# WEB SCRAPING
  
Es una técnica para extraer y almacenar datos de una o varias páginas web con el fin de analizarlos o manipularlos en otros medios, para la cual se utilizan bots para extraer los datos y contenidos de las webs. 

### Importación de librerías.

* Requests realiza la petición al servidor.

* BeautifulSoup analizar documentos HTML.

* Pandas podemos representar datos tabulares con columnas con etiquetas y filas y series temporales.

In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

Se realiza un request de la url_inicial de la revista, url_root nos ayuda a genera un url completa a partir de la url raíz para esto se utiliza la librería urljoin.

In [ ]:
url_inicial = 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/archive'
url_root = 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/archive'
r=requests.get(url_inicial)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')

Obtención de los primeros volúmenes.

Se realiza la búsqueda para obtener los urls de cada uno de los volúmenes o archivos de la revista.

In [ ]:
box = soup.find('div', id='issues')
volumen=box.findAll('div', id='issue')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

Obtención de los primeros volúmenes.

Se realiza la búsqueda para obtener los urls de cada uno de los volúmenes o archivos de la revista.

In [ ]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', id='content') 
    volumen1=box1.findAll('table', class_='tocArticle')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [ ]:
vol2

En esta función variable sopa se utiliza para tener las paginaciones de la revista y la variable url es guardan cada uno de url que ya se obtuvieron de *vol2*.

In [ ]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', id='content') 
    volumen1=box1.findAll('table', class_='tocArticle')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

Ahora se va acumulando cada uno de los links e ir iterando en cada una de las páginas, para traer cada uno de los links que se van a ir scrapeando.

In [ ]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Obtiene el número de los artículos que se encontraron en los primeros volúmenes de la revista.

In [ ]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

Toma uno a uno de los links donde se encuentra la información de cada artículo para Scrapearlos.

In [ ]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

Scrapeo de las urls de los ariculos.

Mediante esta función revisa cada uno de los links de los artículos en los cuales se encuentra la información del artículo, por lo cual se aplican cada uno de los métodos, que nos van a obtener lo que estamos requiriendo de dicho artículo.

In [ ]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Gestión y Política Pública'
    a='V. Ciencias Sociales'
    tem='Administración Pública y Estudio de las Organizaciones'
    s_item=BeautifulSoup(r.text,'html.parser')
 #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('div', id='articleTitle').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', id='articleAbstract').get_text(strip=True)
        content_book['Resumen']=resu.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', id='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='file').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

El list_scraper hace un scrapeo de la cada uno de los artículos, ya que en datos_book se acumuló la información obtenida de los métodos anteriores.

In [ ]:
list_scraper=list_scraper[0:152]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

La variable **datos_book** es un listado en el que se recolecto la información requerida por lo cual se convierte en un **DataFrame**.

In [ ]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

La información solicitada se pasa a un **CSV**.

In [ ]:
df_catalogo.to_csv('Revista065.01.csv', index=False)

Final del Código